In [41]:
library(tidyverse)

# Github Link
https://github.com/drv111llI/dsci100

In [38]:
url <- "https://drive.google.com/uc?id=1Vk02BWAO3Xd_unJJKj0RdpTq1ciLNPO2&export=download"
players <- read_csv(url)
head(players)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


# Data description

The players.csv contains information for 196 participants with 7 columns. 


# Issues with data

2 rows have Age = NA

played_hours is zero-inflated and right-skewed with large outliers

hashedEmail is a privacy-preserving unique id; name is not a reliable identifier.


# Details regarding variables

experience (factor) - User competency level; one of Amateur, Beginner, Pro, Regular, Veteran.

subscribe (logical) - TRUE or FALSE

hashedEmail (chr) - Hashed email used as a unique user identifier.

played_hours (numeric) - Total hours played on the server; numeric

name (chr) - Player nickname on the server; may not be unique.

gender (factor) - Self-reported gender; values include Male, Female, Non-binary, Prefer not to say, Agender, Two-Spirited, Other.

Age (numeric) - Player age in years; numeric

## Question: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

ERROR: Error in eval(expr, envir, enclos): object 'player' not found
